In [15]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine

from nipype.interfaces.utility import Function, IdentityInterface

from nipype.interfaces.fsl.maths import MathsCommand
from nipype.interfaces.fsl.utils import PlotMotionParams   # to plot moco variabl
from nipype import SelectFiles, Node

from ginnipi.toolbox.computations import extract_atlas_ROIs, create_gexf_graph, compute_correlation_matrix
import os
import ginnipi

## AICHA ROI extraction / mean calculation Workflow Start

In [16]:
# creation of a subworflow to process ROI Correlations
connectivity = pe.Workflow(name='connectivity')

In [17]:
bpdir = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/bandpassedFile'
maskdir = '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_files'


from nipype import SelectFiles, Node
templates = dict(bpFile=bpdir+ "/" + "*merged_bp.nii.gz",
                 maskFile=maskdir + "/" + "wc1*.nii")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()

{'bpFile': <undefined>, 'maskFile': <undefined>}

In [18]:
# Node: func2.AICHA_ts_extract  
from ginnipi.toolbox.computations import extract_atlas_ROIs, create_gexf_graph, compute_correlation_matrix

func2_atlas=os.path.join(os.path.dirname(ginnipi.__file__),'share','atlas','AICHA-2mm.nii.gz')                        
func2_xml=os.path.join(os.path.dirname(ginnipi.__file__),'share','atlas','AICHA_Short.xml')

AICHA_ts_extract = Node(Function(input_names=['bpFile', 'mask_file', 'atlas_file', 'atlas_xml','eig'],                                      
                                 output_names=['out_ts_file'],                                      
                                 function=extract_atlas_ROIs), name="AICHA_ts_extract")     
AICHA_ts_extract.inputs.atlas_file = func2_atlas     
AICHA_ts_extract.inputs.atlas_xml = func2_xml     
AICHA_ts_extract.inputs.ignore_exception = False   
# mean calculation on AICHA ROI 
AICHA_ts_extract.inputs.eig=False    
connectivity.connect(filesource, "bpFile", AICHA_ts_extract, "timeseries_file")     
connectivity.connect(filesource, "maskFile", AICHA_ts_extract, "mask_file")   # grey matter mask

In [19]:
# Node: func2.AICHA_cormat
from ginnipi.toolbox.computations import extract_atlas_ROIs, create_gexf_graph, compute_correlation_matrix

AICHA_cormat = Node(Function(input_names=['in_file'], 
                                 output_names=['out_file', 'out_file2'],
                                 function=compute_correlation_matrix), name="AICHA_cormat")
AICHA_cormat.inputs.ignore_exception = False
connectivity.connect(AICHA_ts_extract, "out_ts_file", AICHA_cormat, "in_file")


In [20]:
# output node
outputNode = Node(IdentityInterface(fields=['out_file', 'out_file2']), name="outputNode")

# for correlation matrix
connectivity.connect(AICHA_cormat,  'out_file', outputNode, 'out_file')
connectivity.connect(AICHA_cormat,  'out_file2', outputNode, 'out_file2')

In [21]:
# data sink
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/scratch/user/hirsch/datadir4/data_results_py'

# for motion correction  plot files
connectivity.connect(outputNode,  'out_file', datasink, 'functionnal.aicha_correlaion_matrix')
connectivity.connect(outputNode,  'out_file2', datasink, 'functionnal.@2aicha_correlaion_matrix')

In [22]:
# the RUN

connectivity.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /tmp/tmpFszn9t/connectivity/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node AICHA_ts_extract in dir: /tmp/tmpU35mut/connectivity/AICHA_ts_extract
ERROR:workflow:['Node AICHA_ts_extract failed to run on host pipeau.dgin.bor.']
INFO:workflow:Saving crash info to /homes_unix/hirsch/_ipnotebooks/crash-20170426-115505-hirsch-AICHA_ts_extract-59b8cc58-d8b4-49ff-9c67-153f18fde33b.pklz
INFO:workflow:Traceback (most recent call last):
  File "/srv/shares/softs/anaconda2/lib/python2.7/site-packages/nipype/pipeline/plugins/linear.py", line 39, in run
    node.run(updatehash=updatehash)
  File "/srv/shares/softs/anaconda2/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 394, in run
    self._run_interface()
  File "/srv/shares/softs/anaconda2/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", 

RuntimeError: Workflow did not execute cleanly. Check log for details